In [1]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [2]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsUSB"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="0957", ATTRS{idProduct}=="1798" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

rm = pyvisa.ResourceManager()
rm.list_resources('?*')

inst = rm.open_resource('USB0::2391::6040::MY58103677::0::INSTR')

inst.write('*IDN?')
print(inst.read())

AGILENT TECHNOLOGIES,DSO-X 2014A,MY58103677,02.43.2018020635



In [3]:
'''
set up backend
'''
import importlib
import serverBackend as sb
importlib.reload(sb)

#add the commands we want to use - see manual
osc_keys = [
    'CHANnel1:DISPlay',
    'CHANnel2:DISPlay',
    'CHANnel3:DISPlay',
    'CHANnel4:DISPlay',
    'DIGitize',
    'WAVeform:SOURce', # CHANnel1
    'WAVeform:FORMat',# BYTE
    'WAVeform:POINts', # 500
    'WAVeform:DATA', #?
    'RUN', #?
    'STOP', #?
    'AUToscale',
    ]
sb.setup(inst, osc_keys, 'oscilloscope.log')
sb.query('*IDN?')

*IDN? AGILENT TECHNOLOGIES,DSO-X 2014A,MY58103677,02.43.2018020635



'AGILENT TECHNOLOGIES,DSO-X 2014A,MY58103677,02.43.2018020635'

In [14]:
sb.set_single_param('CHANnel1:DISPlay','ON')
sb.set_single_param('CHANnel2:DISPlay','ON')
sb.set_single_param('CHANnel3:DISPlay','ON')
sb.set_single_param('CHANnel4:DISPlay','OFF')
#sb.query('CHANnel1:DISPlay ON')
sb.set_single_param('WAVeform:SOURce','CHANnel2')
ch = sb.query('WAVeform:DATA?')

   set  CHANnel1:DISPlay: ON
   set  CHANnel2:DISPlay: ON
   set  CHANnel3:DISPlay: ON
   set  CHANnel4:DISPlay: OFF
   set  WAVeform:SOURce: CHANnel2
WAVeform:DATA? #800001399 4.30064e-001, 4.30064e-001, 3.89863e-001, 4.30064e-001, 3.89863e-001, 4.30064e-001, 4.30064e-001, 4.30064e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.89863e-001, 4.30064e-001, 3.89863e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.89863e-001, 3.89863e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.89863e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.89863e-001, 3.49662e-001, 3.49662e-001, 3.89863e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.89863e-001, 3.49662e-001, 4.30064e-001, 3.49662e-001, 3.89863e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.89863e-001, 3.89863e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 3.49662e-001, 4.70265e-001, 4.30064e-001, 4.30064e-001, 4.30064e-001, 4.30064e-001, 4.70265e-001, 4.70265e-001, 4.70265e-00

In [16]:
sb.set_single_param('AUToscale','')
time.sleep(1)
sb.set_single_param('WAVeform:SOURce','CHANnel1')
#sb.set_single_param('WAVeform:FORMat','ASCii')
#sb.set_single_param('WAVeform:POINts','100')
#sb.set_single_param('DIGitize','')
#waveform = sb.query('WAVeform:DATA?')
#print(waveform)
ch = sb.query('WAVeform:DATA?')

sb.set_single_param('WAVeform:SOURce','CHANnel2')
ch = sb.query('WAVeform:DATA?')
sb.set_single_param('RUN','')

   set  AUToscale: 
   set  WAVeform:SOURce: CHANnel1
WAVeform:DATA? #800001399 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.55978e+000, 1.57988e+000, 1.62008e+000, 1.62008e+000, 1.64018e+000, 1.64018e+000, 1.66028e+000, 1.68038e+000, 1.70048e+000, 1.70048e+000, 1.70048e+000, 1.72058e+000, 1.72058e+000, 1.74068e+000, 1.74068e+0

In [17]:
stored_c2 = ''
def get_C1C2():
    sb.set_single_param('AUToscale','')
    time.sleep(1)
    #sb.set_single_param('DIGitize','')
    sb.set_single_param('WAVeform:SOURce','CHANnel1')
    ch1 = sb.query('WAVeform:DATA?')
    sb.set_single_param('WAVeform:SOURce','CHANnel2')
    ch2 = sb.query('WAVeform:DATA?')
    sb.set_single_param('RUN','')
    return ch1+'_'+ch2

def get_C1C2_avg():
    sb.set_single_param('AUToscale','')
    time.sleep(1)
    #sb.set_single_param('DIGitize','')
    sb.set_single_param('WAVeform:SOURce','CHANnel1')
    ch1 = sb.query('WAVeform:DATA?')
    ch1 = np.array([float(s.strip(',')) for s in ch1.split(',')[1:]])    
    sb.set_single_param('WAVeform:SOURce','CHANnel2')
    ch2 = sb.query('WAVeform:DATA?')
    ch2 = np.array([float(s.strip(',')) for s in ch2.split(',')[1:]])
    sb.set_single_param('WAVeform:SOURce','CHANnel3')
    ch3 = sb.query('WAVeform:DATA?')
    ch3 = np.array([float(s.strip(',')) for s in ch3.split(',')[1:]])
    sb.set_single_param('RUN','')
    return str(np.average(ch1))+', '+str(np.average(ch2))+', '+str(np.average(ch3))

def get_C1_avg():
    sb.set_single_param('AUToscale','')
    time.sleep(1)
    #sb.set_single_param('DIGitize','')
    sb.set_single_param('WAVeform:SOURce','CHANnel1')
    ch1 = sb.query('WAVeform:DATA?')
    ch1 = np.array([float(s.strip(',')) for s in ch1.split(',')[1:]])    
    sb.set_single_param('WAVeform:SOURce','CHANnel2')
    ch2 = sb.query('WAVeform:DATA?')
    ch2 = np.array([float(s.strip(',')) for s in ch2.split(',')[1:]])
    sb.set_single_param('RUN','')
    global stored_c2
    stored_c2 = np.average(ch2)
    return str(np.average(ch1))

def get_C2_cor():
    global stored_c2
    print(str(stored_c2))
    return str(stored_c2)

def get_C1_avg():
    return str(-1)

def get_C2_cor():
    return str(-1)

sb.override_dict = {'C1C2?':get_C1C2, 
                    'AvgC1C2?':get_C1C2_avg,  
                    'AvgC1?':get_C1_avg,
                    'AvgC2?':get_C2_cor,
                   }
#sb12 = sb.query('C1C2?')
sb12 = sb.query('AvgC1C2?')
print(sb12)

   set  AUToscale: 
   set  WAVeform:SOURce: CHANnel1
WAVeform:DATA? #800001399 1.79099e+000, 1.81109e+000, 1.81109e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.81109e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.77089e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.81109e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.79099e+000, 1.81109e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.79099e+000, 1.54978e+000, 1.56988e+000, 1.58998e+000, 1.63018e+000, 1.63018e+000, 1.65028e+000, 1.67038e+000, 1.67038e+000, 1.67038e+000, 1.71058e+000, 1.71058e+000, 1.73068e+000, 1.73068e+000, 1.75079e+000, 1.75079e+0

In [ ]:
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = sb.get_ip() #"127.0.0.1"
port = 5007
serversocket.bind((host, port))
serversocket.listen(5)
print ('server started and listening')
while True:
    clientsocket, address = serversocket.accept()
    sb.server(clientsocket, address)

server started and listening
Current Time = 20:26:14
   set  AUToscale: 
   set  WAVeform:SOURce: CHANnel1
WAVeform:DATA? #800001399 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.82109e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.80099e+000, 1.78089e+000, 1.78089e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.80099e+000, 1.78089e+000, 1.78089e+000, 1.80099e+000, 1.78089e+000, 1.78089e+000, 1.80099e+000, 1.53968e+000, 1.57988e+000, 1.59998e+000, 1.62008e+000, 1.64018e+000, 1.66028e+000, 1.66028e+000, 1.68038e+000, 1.70048e+000, 1.70048e+000, 1.70048e+000,